In [1]:
!pip install fastapi uvicorn oracledb pandas nest_asyncio scikit-learn

In [2]:
from fastapi import FastAPI
# oracledb: 오라클 데이터베이스와 연결해서 데이터를 가져오거나 저장하는 도구입니다.
import oracledb
# pandas(pd): 데이터프레임을 쉽게 만들고 다룰 수 있게 해주는 도구입니다.
import pandas as pd
# nest_asyncio: fastapi를 주피터 노트북처럼 여러 셀에서 사용할 수 있게 해주는 도구입니다.
import nest_asyncio
# uvicorn: FastAPI 애플리케이션을 띄우는 도구입니다.
import uvicorn
# pickle: 데이터를 쉽게 저장하거나 다시 읽을 수 있게 해주는 도구입니다.
import pickle
# cosine_similarity: 데이터의 유사도를 비교하고 계산할 수 있는 도구입니다.
from sklearn.metrics.pairwise import cosine_similarity
# BaseModel: 데이터 검증과 모델링을 도와주는 도구입니다.
from pydantic import BaseModel
# numpy(np): 수치 연산을 빠르고 쉽게 해주는 도구입니다.
import numpy as np
# CORS Middleware: 외부 클라이언트가 만든 요청을 서버에 허용할 수 있도록 설정해주는 도구입니다.
from fastapi.middleware.cors import CORSMiddleware

In [3]:
app = FastAPI()

In [4]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
@app.get("/movie/hello_backend")
async def hello():
    return "안녕 FastAPI"

In [6]:
userdsn  = oracledb.makedsn("localhost",1521,service_name="XEPDB1")
username = "MOVIE_ADMIN"
userpass= "pass123"

In [7]:
@app.get("/movie/movie_popular")
async def movie_popular():
    connection = oracledb.connect(user=username, password=userpass, dsn=userdsn)

    query = """
        SELECT num, title, director
                ,actor, box_office
                ,synopsis, poster
                ,open_date, degree
                ,country, movie_time
        From movie_tbl
        ORDER BY box_office DESC
        FETCH FIRST 30 ROWS ONLY
    """

    movie_df = pd.read_sql(query, con=connection)
    connection.close()

    return movie_df.to_dict(orient='records')

In [8]:
nest_asyncio.apply()

In [ ]:
uvicorn.run(app, host="0.0.0.0", port=7070)

INFO:     Started server process [25148]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7070 (Press CTRL+C to quit)


INFO:     127.0.0.1:51046 - "GET /movie/hello_backen HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51048 - "GET /movie/hello_backend HTTP/1.1" 200 OK
INFO:     127.0.0.1:51056 - "GET /movie/movie_popular HTTP/1.1" 200 OK


C:\Users\User\AppData\Local\Temp\ipykernel_25148\629524516.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(query, con=connection)
